In [1]:
import numpy as np
import pymc
import pdb

In [2]:
def unconditionalProbability(Ptrans):
   """Compute the unconditional probability for the states of a
   Markov chain."""

   m = Ptrans.shape[0]

   P = np.column_stack((Ptrans, 1. - Ptrans.sum(axis=1)))

   I = np.eye(m)
   U = np.ones((m, m))
   u = np.ones(m)

   return np.linalg.solve((I - P + U).T, u)

data = np.loadtxt('test_data.txt',
                 dtype=np.dtype([('state', np.uint8),
                                 ('emission', np.float)]),
                 delimiter=',',
                 skiprows=1)

# Two state model for simplicity.
N_states = 2
N_chain = len(data)

# Transition probability stochastic
theta = np.ones(N_states) + 1.

def Ptrans_logp(value, theta):
   logp = 0.
   for i in range(value.shape[0]):
       logp = logp + pymc.dirichlet_like(value[i], theta)
   return logp

def Ptrans_random(theta):
   return pymc.rdirichlet(theta, size=len(theta))

Ptrans = pymc.Stochastic(logp=Ptrans_logp,
                        doc='Transition matrix',
                        name='Ptrans',
                        parents={'theta': theta},
                        random=Ptrans_random)

#Hidden states stochastic
def states_logp(value, Ptrans=Ptrans):
    
    if sum(value>1):
        return -np.inf

    P = np.column_stack((Ptrans, 1. - Ptrans.sum(axis=1)))

    Pinit = unconditionalProbability(Ptrans)

    logp = pymc.categorical_like(value[0], Pinit)

    for i in range(1, len(value)):
       try:
          logp = logp + pymc.categorical_like(value[i], P[value[i-1]])
       except:
          pdb.set_trace()

    return logp

def states_random(Ptrans=Ptrans, N_chain=N_chain):
   P = np.column_stack((Ptrans, 1. - Ptrans.sum(axis=1)))

   Pinit = unconditionalProbability(Ptrans)

   states = np.empty(N_chain, dtype=np.uint8)

   states[0] = pymc.rcategorical(Pinit)

   for i in range(1, N_chain):
       states[i] = pymc.rcategorical(P[states[i-1]])

   return states

states = pymc.Stochastic(logp=states_logp,
                        doc='Hidden states',
                        name='states',
                        parents={'Ptrans': Ptrans},
                        random=states_random,
                        dtype=np.uint8)

# Gaussian emission parameters
mu = pymc.Normal('mu', 0., 1.e-6, value=np.random.randn(N_states))
sigma = pymc.Uniform('sigma', 0., 100.,
value=np.random.rand(N_states))

y = pymc.Normal('y', mu[states], 1./sigma[states]**2,
value=data['emission'], observed=True)

In [3]:
M = pymc.MCMC([Ptrans, states, mu, sigma, y])

In [4]:
M.sample(iter=100000, burn=10000, thin=10)

 [-----------------100%-----------------] 100000 of 100000 complete in 6483.2 sec

In [5]:
M.summary()


mu:
 
	Mean             SD               MC Error        95% HPD interval
	------------------------------------------------------------------
	1.007            0.011            0.001            [ 0.983  1.028]
	-0.335           0.263            0.026              [-0.67  0.1 ]
	
	
	Posterior quantiles:
	
	2.5             25              50              75             97.5
	 |---------------|===============|===============|---------------|
	0.982            1.0             1.008          1.015         1.027
	-0.664           -0.593          -0.406         -0.077        0.115
	


/Users/hstrey/anaconda/envs/python3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:225: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return reshape(newshape, order=order)



states:
 
	Mean             SD               MC Error        95% HPD interval
	------------------------------------------------------------------
	0.714            0.452            0.042                  [ 0.  1.]
	0.12             0.325            0.03                   [ 0.  1.]
	0.093            0.29             0.027                  [ 0.  1.]
	0.0              0.0              0.0                    [ 0.  0.]
	0.109            0.312            0.03                   [ 0.  1.]
	0.149            0.356            0.035                  [ 0.  1.]
	0.026            0.16             0.015                  [ 0.  0.]
	0.073            0.26             0.026                  [ 0.  1.]
	0.014            0.119            0.01                   [ 0.  0.]
	0.0              0.0              0.0                    [ 0.  0.]
	0.045            0.207            0.018                  [ 0.  0.]
	0.038            0.192            0.018                  [ 0.  0.]
	0.029            0.169            0.

In [6]:
myP=np.array([[0.9],[0.3]])
unconditionalProbability(myP)

array([ 0.75,  0.25])

In [9]:
Ptrans_random(theta)

array([[ 0.46391414],
       [ 0.35178629]])